In [1]:
import argparse
import glob
import random
import numpy as np
! pip install scikit-learn --quiet
! pip install cython --quiet
! pip install h5py --quiet
from sklearn.svm import SVC
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F

from datasets.data import *
from datasets.plyfile import load_ply
from models.dgcnn import DGCNN, ResNet, DGCNN_partseg

parser = argparse.ArgumentParser(description='Point Cloud Recognition')
parser.add_argument('--num_points', type=int, default=1024,
                    help='num of points to use')
parser.add_argument('--emb_dims', type=int, default=1024, metavar='N',
                    help='Dimension of embeddings')
parser.add_argument('--k', type=int, default=15, metavar='N',
                        help='Num of nearest neighbors to use')
parser.add_argument('--dropout', type=float, default=0.5,
                        help='dropout rate')
args = parser.parse_args("")

/Users/emanuelerucci/opt/anaconda3/envs/mycondaenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
if (torch.cuda.is_available()):
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
        
net_self = torch.load('dgcnn_cls_best.pth')
model_self = DGCNN(args).to(device)

model_self.load_state_dict(net_self)

FileNotFoundError: [Errno 2] No such file or directory: 'dgcnn_cls_best.pth'

# ModelNet40

In [5]:
train_loader = DataLoader(ModelNet40SVM(partition='train', num_points=args.num_points),
                              batch_size=128, shuffle=True)
test_loader = DataLoader(ModelNet40SVM(partition='test', num_points=args.num_points),
                              batch_size=128, shuffle=True)
print('Done !!')

ValueError: need at least one array to concatenate

# ScanObjectNN

In [6]:
train_loader = DataLoader(ScanObjectNNSVM(partition='train', num_points=args.num_points),
                              batch_size=64, shuffle=True)
test_loader = DataLoader(ScanObjectNNSVM(partition='test', num_points=args.num_points),
                              batch_size=64, shuffle=True)
print('Done !!')

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'data/ScanObjectNN/main_split/train.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

# Load Train Feats

In [10]:
import random

feats_train = []
labels_train = []
model = model_self.to(device)
model = model.eval()

dataset = "ScanObjectNN" # Choose Dataset ["ModelNet40, ScanObjectNN"]

for i, (data, label) in enumerate(train_loader):
    if dataset == "ModelNet40":
        labels = list(map(lambda x: x[0],label.numpy().tolist()))
    elif dataset == "ScanObjectNN":
        labels = label.numpy().tolist()
    data = data.permute(0, 2, 1).to(device)
    with torch.no_grad():
        feats = model(data)[2]
    feats = feats.detach().cpu().numpy()
    for feat in feats:
        feats_train.append(feat)
    labels_train += labels

feats_train = np.array(feats_train)
labels_train = np.array(labels_train)
print(feats_train.shape)
print(labels_train)

(2309, 2048)
[ 7 13  7 ...  3  7 10]


# Load Test Feats

In [11]:
import random

feats_test = []
labels_test = []
model = model_self.to(device)
model = model.eval()

for i, (data, label) in enumerate(test_loader):
    if dataset == "ModelNet40":
        labels = list(map(lambda x: x[0],label.numpy().tolist()))
    elif dataset == "ScanObjectNN":
        labels = label.numpy().tolist()
    data = data.permute(0, 2, 1).to(device)
    with torch.no_grad():
        feats = model(data)[2]
    feats = feats.detach().cpu().numpy()
    for feat in feats:
        feats_test.append(feat)
    # labels_train.append(label)
    labels_test += labels

feats_test = np.array(feats_test)
labels_test = np.array(labels_test)
feats_test.shape

(581, 2048)

# Train Linear Model

In [12]:
c = 0.01 # Linear SVM parameter C, can be tuned
model_tl = SVC(C = c, kernel ='linear')
model_tl.fit(feats_train, labels_train)
print(f"C = {c} : {model_tl.score(feats_test, labels_test)}")

C = 0.01 : 0.8123924268502581
